<a href="https://colab.research.google.com/github/lily02092003/ml-project/blob/main/salesanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -Uq upgini catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.6 MB/s eta 0:00:00


PREPARING THE DATA

In [ ]:
from os.path import exists
import pandas as pd
df_path= "train.csv.zip"
df= pd.read_csv(df_path)
df=df.sample(n=19000,random_state=0)
df["store"]=df["store"].astype(str)
df["item"]=df["item"].astype(str)
df["date"]=pd.to_datetime(df["date"])
df.sort_values("date",inplace=True)
df.reset_index(inplace=True,drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [ ]:
train= df[df["date"]<"2017-01-01"]
test= df[df["date"]>="2017-01-01"]

In [ ]:
train_features=train.drop(columns="sales")
train_target=train["sales"] #predicting sales column
test_features=test.drop(columns="sales")
test_target=test["sales"] #predicting sales column

ENRICH FEATURES

In [ ]:
from upgini import FeaturesEnricher,SearchKey
from upgini.metadata import CVType
enricher=FeaturesEnricher(
    search_keys={
        "date": SearchKey.DATE,
    },
    cv=CVType.time_series
)
enricher.fit(train_features,train_target,eval_set=[(test_features,test_target)])


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history
Detected task type: ModelTaskType.REGRESSION



Column name,Status,Errors
date,All valid,-
target,All valid,-



Running search request, search_id=8711487a-97a5-4849-b8e0-1fba05871b89
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


30 relevant feature(s) found with the search keys: ['date']


,,item,0.593929,100.000000,categorical,
,,store,0.195910,100.000000,categorical,
Upgini,Public data,f_weather_date_weather_umap_31_fa6d9a99,0.036087,100.000000,numerical,Free
Upgini,Public data,f_events_date_week_sin1_847b5db1,0.033563,100.000000,numerical,Free
Upgini,Public data,f_events_date_year_cos1_9014a856,0.032180,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_48_b39cd0c4,0.030332,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_47_5123ef0a,0.014408,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_34_c3ef5b4f,0.013787,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_pca_0_d7e0a1fc,0.013082,100.000000,numerical,Free
Upgini,Public data,f_events_date_week_cos3_7525fe31,0.009484,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_33_89bb7578,0.004414,100.000000,numerical,Free


We detected 113 outliers in your sample.
Examples of outliers with maximum value of target:
84    205
47    196
38    187
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Before dropping target outliers size: 19000
After dropping target outliers size: 18887
Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.

Quality metrics


,Rows,Baseline mean_squared_error,Enriched mean_squared_error,Uplift
,,,,
Train,15148,309.990498,195.876523,114.113975
Eval 1,3739,509.289740,366.309441,142.980299


In [ ]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric
model=CatBoostRegressor(verbose=False,allow_writing_files=False,random_state=0)
enricher.calculate_metrics(
    train_features,train_target,
    eval_set=[(test_features,test_target)],
    estimator=model,
    scoring="mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
-
which makes metrics between the train and eval_set incomparable.


,Rows,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
,,,,
Train,15148,0.255109,0.155266,0.099843
Eval 1,3739,0.270377,0.194607,0.075770


In [ ]:
enriched_train_features=enricher.transform(train_features,keep_input=True)
enriched_test_features=enricher.transform(test_features,keep_input=True)
enriched_train_features.head()


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=8960254e-31b0-49ee-9c27-f00439e8b3f5
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=dcb929a6-af32-4263-bfa5-96760459318c
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


,date,store,item,f_weather_date_weather_umap_31_fa6d9a99,f_events_date_week_sin1_847b5db1,f_events_date_year_cos1_9014a856,f_weather_date_weather_umap_48_b39cd0c4,f_weather_date_weather_umap_47_5123ef0a,f_weather_date_weather_umap_34_c3ef5b4f,f_weather_date_weather_pca_0_d7e0a1fc,...,f_economic_date_cci_pca_3_10646e17,f_economic_date_cci_pca_6_aa7c1005,f_events_date_year_sin1_3c44bc64,f_weather_date_weather_umap_12_d03be9a0,f_economic_date_cbpol_pca_1_31e5f62c,f_financial_date_finance_umap_0_526a7a88,f_weather_date_weather_umap_35_5ddaa0ba,f_weather_date_weather_umap_22_0342ee9e,f_economic_date_cpi_umap_7_d43e2396,f_weather_date_weather_umap_45_d474bf8d
0,2013-01-01,7,5,4.712653,0.781831,0.98522,4.540985,5.927147,5.664261,29.676683,...,-1.962578,-1.387072,0.171293,3.739055,-0.438029,10.955449,4.76773,4.806711,11.154531,4.923654
1,2013-01-01,4,9,4.712653,0.781831,0.98522,4.540985,5.927147,5.664261,29.676683,...,-1.962578,-1.387072,0.171293,3.739055,-0.438029,10.955449,4.76773,4.806711,11.154531,4.923654
2,2013-01-01,1,33,4.712653,0.781831,0.98522,4.540985,5.927147,5.664261,29.676683,...,-1.962578,-1.387072,0.171293,3.739055,-0.438029,10.955449,4.76773,4.806711,11.154531,4.923654
3,2013-01-01,3,41,4.712653,0.781831,0.98522,4.540985,5.927147,5.664261,29.676683,...,-1.962578,-1.387072,0.171293,3.739055,-0.438029,10.955449,4.76773,4.806711,11.154531,4.923654
4,2013-01-01,5,24,4.712653,0.781831,0.98522,4.540985,5.927147,5.664261,29.676683,...,-1.962578,-1.387072,0.171293,3.739055,-0.438029,10.955449,4.76773,4.806711,11.154531,4.923654


In [ ]:
model.fit(enriched_train_features,train_target)
preds=model.predict(enriched_test_features)
eval_metric(test_target.values,preds,"SMAPE")

[14.681142656550785]